# First-Year Writing Seminars: Extension

**Objectives:**
* Formulate a new approach to the FWS assignment problem using ideas from min-cost flow.
* Discuss how an integer program can be improved by updating decision variables, constraints, and the objective function.
* Get comfortable with ORTools syntax.

**Key Ideas:**
* integrality property
* the min-cost flow problem
* the transportation problem
* the assignment problem

**Reading Assignment:**
* Read Handout 7.5 on the min-cost flow problem.
* Read through the FWS lab to refresh your memory on key concepts.

**Brief description:** If you recall pre-enroll, there was a separate ballot you completed by listing your top 5 picks for FWS that semester. You were later notified which class you got placed into, probably hoping it was your first choice. By now, this should not seem like magic; problems like these often enlist help from Operations Research especially as the scale increases. Disclaimer: the following models are not actually used by Cornell.

In [ ]:
# imports -- don't forget to run this cell!
import numpy as np
import pandas as pd
import math, itertools
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp as OR
from ortools.graph import pywrapgraph as ORMC
from fws_lab_ext import inputData

# Part 0: A Quick Recap

In the FWS lab, we dealt with an *assignment problem*: we were trying to assign students to different First-Year Writing Seminar sections, given a list of each student's preferences. This is a special case of the *transportation problem*, where we want to ship units from supply nodes to demand nodes. (Here, we are "shipping" students to classes!)

A small input for such a problem might be as follows: we have 4 classes (ABCD) that can each hold 2 students. There are 7 students available, whose preferred classes are listed in the table below.

| Student | First | Second |
|:-------:|:-----:|:------:|
|    1    |   A   |    B   |
|    2    |   D   |    C   |
|    3    |   A   |    C   |
|    4    |   B   |    D   |
|    5    |   C   |    B   |
|    6    |   A   |    B   |
|    7    |   B   |    A   |

To solve this problem, we might set up a graph like the following:

In [ ]:
from fws_lab_ext import small_ex

S = [1,2,3,4,5,6,7]
D = ['A','B','C','D']
E = {(1,'A'):1, 
     (1,'B'):2, 
     (2,'D'):1, 
     (2,'C'):2, 
     (3,'A'):1, 
     (3,'C'):2, 
     (4,'B'):1, 
     (4,'D'):2, 
     (5,'C'):1, 
     (5,'B'):2, 
     (6,'A'):1, 
     (6,'B'):2, 
     (7,'B'):1, 
     (7,'A'):2 }

small_ex(S, D, E)

The supply nodes (representing students) are on the left, and the demand nodes (representing classes) are on the right. A directed edge $(i,j)$ from a supply node $i$ to a demand node $j$ means that we can "send" (assign) student $i$ to class $j$, at some unit cost $c[i,j]$. Note that "first-choice" edges are in blue, and "second-choice" edges are in orange.

**Q0:** Right now, we have a demand of $(4)(2) = 8$ students, but we can only supply 7 students. What additional nodes and edges do we need to include in our graph to make sure we can satisfy our demand?

**A:** <font color='blue'>Include a "dummy" supply node that has arcs going from it to every demand node. </font>

Now our graph looks something like this:

In [ ]:
S_dummy = S + ['dummy']
D_dummy = D
E_dummy = dict(E)
E_dummy.update({('dummy','A'):3,
                ('dummy','B'):3,
                ('dummy','C'):3,
                ('dummy','D'):3})

small_ex(S_dummy,D_dummy,E_dummy)

If we drew a graph for the actual FWS input, it would look similar to this, except with thousands of student supply nodes and hundreds of class demand nodes! 

As a reminder, in the real-world problem, each student gives up to 5 preferences and each class section is capped at 17 students.

# Part 1: Min-Cost Flow Formulation

**Review** 

Recall the min-cost flow problem. It takes as input
* A directed graph $G = (V,A)$,
* costs $c(i,j)$ for shipping one unit of good from node $i$ to node $j$ for each arc $(i,j) \in A$,
* capacities $u(i,j)$ for each arc $(i,j) \in A$,
* supply values $b(i)$ for each node $i \in V$, such that $\sum_{i \in V} b(i) = 0$.

Remember also that at each node $i$, our supply value $b(i)$ is greater than 0 if there is supply at node $i$, less than 0 if there is demand at node $i$, and equal to 0 if there is neither supply nor demand at node $i$ (i.e., node $i$ is a transit node). Using max-flow terminology, supply nodes are "sources," demand nodes are "sinks," and transit nodes are interior nodes.

Our goal is to find a feasible flow that satisfies both flow-capacity constraints and flow-conservation constraints; that is, we wish to find a flow $f(i,j)$ on all arcs such that $0 \leq f(i,j) \leq u(i,j)$ for every arc $(i,j) \in A$ and $\sum_{(i,j) \in A} f(i,j) - \sum_{(j,i) \in A} f(j,i) = b(i)$ for every node $i \in V$.

The objective value of a feasible solution is given by $\sum_{(i,j) \in A} c(i,j)*f(i,j)$. We'd like to minimize this cost function&mdash;in other words, find a "min-cost" flow.

(For a more in-depth discussion, see Handout 7.5 and the min-cost flow lab.)

**Formulating the model**

In Handout 7.5, we learned that the transportation problem is really just a specific case of the min-cost flow problem. Let's use this fact, along with the transportation model we've already created in the FWS lab, to formulate a min-cost flow model. As we'll see, using a min-cost flow approach allows us to incorporate new information into our model.

Nodes for each student and class section, as well as the special 'dummy' supply node, remain the same as before, as do our arcs and edge costs; all we need to do to create a min-cost flow input is define (1) the arc capacities and (2) the supply values at each node, and we'll be all set!

**Q1:** What should the capacity $u(i,j)$ on each arc $(i,j)$ be? (An arc $(i,j)$ connects a student node $i$ to a class node $j$.)

**A:** <font color='blue'>1</font>

**Q2:** We also need to define the capacity $u(dummy,j)$ on each arc leaving the 'dummy' node to a class node. What is the maximum number of dummy students we can send to each class?

**A:** <font color='blue'> Set $u(dummy,j) = 17$. More generally, we set $u(dummy,j) = $ (max number of 'real' students) $ - $ (min number of 'real' students), or (in this case) $17 - 0 = 17$.</font>

Next, let's define our supply values $b(i)$.

**Q3:** For a student node $i$, what should the supply value $b(i)$ be?

**A:** <font color='blue'>1</font>

**Q4:** For a class node $j$, what should the supply value $b(j)$ be? (If there is demand at a node $k$, then $b(k) < 0$.)

**A:** <font color='blue'>-17</font>

Once again, we must account for our dummy supply node. Recall that for a min-cost flow input to be valid, the "net supply/demand" summed up over all nodes should be equal to 0:  $\sum_{i \in V} b(i) = 0$. 

Suppose we have $n$ students selecting from $m$ classes, each of which can have up to 17 students. 

**Q5:** Using this information, what should the supply value $b(dummy)$ be?

**A:** <font color='blue'>To satisfy our input condition $\sum_{i \in V} b(i) = 0$, we must have $\sum_{students,i} b(i)$ + $\sum_{classes,j} b(j)$ + $b(dummy) = 0$. Thus $n(1) - m(17) + b(dummy) = 0$, which gives $b(dummy) = 17m - n$.</font>

This should make sense intuitively; essentially, we are saying that after every student has been assigned a class, whatever spots are left over should be filled by our "fake students." (Of course, this assumes there are enough spots for every "real" student!)

We now have everything we need to formulate the model. As a reminder, we set the unit cost of an edge $(i,j)$ from a student node to the class node representing their $k$th preference to be $k$. For "dummy" edges, we set the unit cost to be an arbitrarily large number (100,000) to discourage the solver from sending flow across those edges unless it absolutely has to.

Run the cell below, which implements our min-cost flow model in Python. (You can read through the code if you'd like.)

In [ ]:
# ORTools min-cost flow implementation  
# Based off of https://developers.google.com/optimization/flow/mincostflow
#
# 'dataset' is the name of the datafile
# 'csize' is the desired class size, filled with a combination of real and 'filler' students
# 'minstudents' is the minimum number of (real) students that must be assigned to each section (between 0 and csize)
# 'dcost' is the cost of not assigning a student to one of their top 5 preferences (i.e., cost of dummy edge)
def mincostflow(dataset='s21_fws_ballots.csv', csize=17, minstudents=0, dcost=100000):
    
    if minstudents > csize or minstudents < 0:
        raise ValueError('Error: minstudents must be in [0,class size].')
    
    students, classes, edges = inputData(dataset)
           
    n = len(students) # number of students
    m = len(classes) # number of class sections
    dcapacity = csize - minstudents # number of dummy students we can send to each class
    
    # define supply b[i] at each node i
    # ORTools spec says nodes must be nonnegative integers indexed starting at 0 (dummy supply node),
    # so class numbers are indexed from (n + 1) to (n + m), where 
    # n is the number of students and m is the number of class sections  
    supplies = []
    supplies.append(csize*m - n) # dummy supply node
    for s in students:
        supplies.append(1) # each student node has supply 1
    for c in classes:
        supplies.append(-1*csize) # each class node has supply -csize (i.e., demand csize)

    # define parallel arrays, one index per arc in the min-cost flow graph 
    start_nodes = []
    end_nodes = []
    capacities = []
    unit_costs = []
    
    # add student edges    
    for i,j in edges:
        start_nodes.append(i) # arcs start at student node 
        end_nodes.append(j+n) # arcs end at class node
        capacities.append(1) 
        unit_costs.append(edges[i,j])
        
    # add dummy edges
    for j in classes:
        start_nodes.append(0)
        end_nodes.append(j+n)
        capacities.append(dcapacity)
        unit_costs.append(dcost)
    
    # create solver
    min_cost_flow = ORMC.SimpleMinCostFlow()
    
    # add arcs, capacities, unit costs to graph
    for i in range(0, len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(int(start_nodes[i]), int(end_nodes[i]), capacities[i], unit_costs[i])  
    
    # add node supplies to graph
    for i in range(0,len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])

    return min_cost_flow

In [ ]:
m = mincostflow()

from fws_lab_ext import printmcf
printmcf(m) # helper function to print results

Success! If everything ran properly, you should now have a working min-cost flow formulation for the FWS assignment problem.

**Q6:** Use the preference list above to calculate the overall cost of the solution. (There are 141 class sections in total, each with a capacity of 17. The cost of a student receiving their $k$th preference is $k$ and the cost of assigning a dummy student to a class section is 100,000.)

**A:** <font color='blue'>The total class capacity is $(141\:sections)(17\:\frac{students}{section}) = 2397$ "spots," of which $1182 + 629 + 299 + 123 + 52 = 2285$ are filled by "real" students and $2397 - 2285 = 112$ are filled by "dummy" students. Now the overall cost is just the summation of the flow on each edge type times the unit cost of that edge type: 
$(1182)(1) + (629)(2) + (299)(3) + (123)(4) + (52)(5) + (112)(100,000) = 11,204,089.$

You can verify this by adding the following lines to the code cell above:<br><code>m.Solve()</code><br><code>print(m.OptimalCost())</code></font>

This is all well and good, but so far we've just repeated what the transportation model already found. What more can we do with min-cost flow?

You may have noticed that our formulation is fairly simple in terms of its assumptions. For example, based off your answer to **Q2**, a feasible (though expensive) solution might involve assigning 17 fake 'filler' students to a section! It's also easy to imagine our model assigning just one or two "real" students to a less interesting section that doesn't rank as high on people's preferences.

The folks at the Knight Institute want to avoid the administrative headaches of a class with just one or two students, while at the same time ensuring students take full advantage of the diversity of FWS classes offered. So, they request that each class section have a minimum of six students enrolled, but no more than 17 (as before).

**Q7:** We need to find a way to "force" our model to assign six real (that is, not filler) students to each class. How can we implement this "minimum class size constraint"? (Hint: take a look at **Q2**)

**A:** <font color='blue'>(taken from answer to Q2) Set $u(dummy,j) = $ (max number of 'real' students) $ - $ (min number of 'real' students), or (in this case) $17 - 6 = 11$. Now we can only send at most 11 dummy students to each class node, but since each class node has a demand of 17, we must fill at least 6 spots with real students.</font>

If you read through the Python function, you may have noticed that it can take as input a parameter called 'minstudents', which specifies the minimum number of "real" students assigned to each class section. (The code generalizes what you did in **Q7**.)

In [ ]:
m = mincostflow(minstudents=0)
printmcf(m)

**Q8:** Try a few different values for the 'minstudents' parameter and see what outputs you get. What do you observe? Can the Knight Institute satisfy their minimum class size constraint? (If not, how might cancelling some classes help?) 

**A:** <font color='blue'>Should see that a feasible flow only exists if the value of minstudents is 0 or 1. The input becomes infeasible for values of minstudents from 2 to 17. Thus the Knight Institute can't satisfy the minimum class size constraint (6) without cancelling classes, as we'll see. Cancelling classes with low interest mitigates this problem by effectively forcing students into more popular classes. </font>

Run the following cell, which outputs the least popular class (or classes) among students' preferences. (Define "least popular" as appearing the least on students' list of preferences.) If you'd like, read the comments alongside each line of code to understand what the function does.

In [ ]:
# Outputs the least popular FWS class section among students' listed preferences
def leastpopular(dataset='s21_fws_ballots.csv'):
    data = pd.read_csv(dataset) # reads in dataset
    
    a = data[['1','2','3','4','5']].values.tolist() # creates a list of all the class preferences students put
    a = [x for x in a if x != 0] # deletes preferences left blank
    unique, counts = np.unique(np.array(a), return_counts=True) # counts how many of each class appears on the preference list
    classdict = dict(zip(unique, counts)) # creates a dictionary of class number : number of preferences
    
    least_students = min(classdict.values()) # finds the minimum number of preferences in the dictionary
    res = [c for c in classdict if classdict[c] == least_students] # finds class number corresponding to min number of prefs.
    
    print('The class (or classes) with the least students interested is ' + str(res) + '.')
    print('Only ' + str(least_students) + ' student(s) put this class as one of their top 5 preferences.') # prints results 
    
leastpopular()

**Q9:** Does this output make sense based on what you observed in **Q8**? Explain.

**A:** <font color='blue'>The function output states that the class with the minimum number of students putting it as a preference has only 1 student interested. Thus only 1 "real" student can be assigned this class--so if we set 'minstudents' higher than 1, there aren't enough students interested to satisfy the minimum class size constraint, and the mincostflow function returns 'Infeasible'.</font>

To summarize, we started with a transportation model for the FWS assignment problem. Then we translated this into a more flexible min-cost flow model, which allowed us to incorporate new information (the Knight Institute's minimum class size constraint) into our formulation. 

In the future, we might want to add even more flexibility, such as the capability to cancel class sections. (Stay tuned!)

# Part 2: A Better Integer Program

In our min-cost flow model for the FWS assignment problem, we discovered that adding a new constraint to our model (that every class must have at least 6 "live" students) made the input infeasible because at least one class didn't have enough students interested. 

We notify the Knight Institute that there seems to be a lack of enthusiasm for class section 1. (Maybe it's at 8 AM on Monday, who knows?) They inquire if they can cancel the section altogether and still find a full matching of students to FWS sections. 

Of course, this is not the only time where being able to decide which class sections actually get run might be helpful. What if an instructor got sick before the school year, or the FWS budget decreased and some class offerings were cut? 

Let's see how we can build this idea into our existing model from the FWS lab (copy/pasted below). 

As a reminder, we formulated this model as an integer program. We have:
* decision variables $x[i,j]$ representing the amount of "student flow" going from student $i$ to class $j$
* a matrix of costs $c[i,j]$ representing the "cost" of assigning students to each of their various class preferences (with better preferences costing less)
* constraints to ensure each class is full, and no student is assigned multiple classes
* additional decision variables $x[dummy,j]$ to fill leftover spots in classes with fake "filler" students if needed, with a very high edge cost $c[dummy,j]$ to make sure the solver fills classes with actual students first
* an objective function to minimize the total cost of the "flow"

In [ ]:
# An FWS assignment model

# INPUTS:
# students: a list of students
# classes: a list of classes
# edges: a dictionary of edge costs
# csize: the class capacity 
# dcost: the cost of not assigning a student to one of their top 5 picks
def Assign(students, classes, edges, csize, dcost, solver):
    STUDENT = students + ['dummy']  # create student list add dummy node 
    CLASS = classes                 # create class list
    EDGES = list(edges.keys())      # create edge list
    
    newedges = list(itertools.product([0], CLASS))
    EDGES.extend(newedges)          # add dummy edges
    
    c = edges.copy()                # define c[i,j]
    for edge in newedges:
        c.update({edge : dcost})    # add c[i,dummy] costs
    
    # define model
    m = OR.Solver('assignFWS', solver)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        # define x(i,j) here
        x[i,j] = m.IntVar(0, m.infinity(), ('(%d, %s)' % (i,j))) 
        
    # define objective function here
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
       
    # add constraint to ensure each student (not including the dummy) is assigned to at most one class
    # this ensures the model won't break in scenarios where matching every student is impossible
    for k in students:
        if k != 'dummy':
            m.Add(sum(x[i,j] for i,j in EDGES if i==k) <= 1)
        
    # add constraint to ensure each class is full
    for k in classes:
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) == csize)
    
    m.Solve()
    
    unmatched = []
    for k in STUDENT:
        if (sum(x[i,j].solution_value() for i,j in EDGES if i==k) == 0) and (k!='dummy'):
            unmatched.append(k)
    print("Unmatched students:", len(unmatched))
    
    matched = {}
    for i,j in EDGES:
        if x[i,j].solution_value() == 1:
            if c[i,j] in matched:
                matched[c[i,j]] += 1
            else:
                matched.update({c[i,j] : 1})
    if dcost in matched.keys():
        del matched[dcost]
    
    return matched

In [ ]:
# read in the dataset
dataset = 's21_fws_ballots.csv' # Spring 2021: 2285 students, 141 class sections
data = pd.read_csv(dataset,index_col=0)
data.head() # preview

In [ ]:
# format data into lists of student and class nodes and a dictionary of edges to edge costs
students, classes, edges = inputData(dataset)

# solve the instance
data_sol = Assign(students, classes, edges, 17, 100000, OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
print(data_sol)

Notice that the <code>Assign</code> function above doesn't include any notion of the minimum class size (6) that the Knight Institute wants! In terms of class size, we just have a constraint specifying that the number of students (real, dummy, or both) in each class must equal <code>csize</code>, in this case 17.

**Q10:** Let's say we replace the constraint mentioned above with one that is a bit more lenient: each class must have between 6 and 17 students. Do we still need "dummy" students in our model? Why? 

**A:** <font color='blue'>The purpose of including the dummy students at all is to satisfy the constraint that every class must be totally filled to capacity with (real and/or fake) students, by backfilling classes that don't have enough real students. By relaxing this constraint, though, there's really no need to include the dummy student supply node, since each class doesn't have to be full as long as at least 6 (real) students are assigned to it. If a class has less than 6 (real) students assigned to it, it doesn't make sense for us to add dummy students until there are 6 students total, because that doesn't solve the problem of lack of interest. Instead, we should just return infeasible in that case.</font>

**Q11:** We'd like to update our integer program to account for whether or not a class section runs. To do so, we'll need to define new binary decision variables. What are they?

**A:** <font color='blue'>Add a new binary decision variable for each class, set to 1 if the class runs and 0 if the class does not run.</font>

**Q12:** Suppose we have a class section A that we'd like to run. What are the upper and lower bounds on the number of students we can assign to class A?

**A:** <font color='blue'>Upper bound = 17; lower bound = 6.</font>

**Q13:** Suppose we have a class section B that we do not want to run. What are the upper and lower bounds on the number of students we can assign to class B?

**A:** <font color='blue'>Upper bound = 0; lower bound = 0.</font>

**Q14:** Suppose we have a class section C and a binary variable $y$, such that if class C runs, $y = 1$, and if class C does not run, $y = 0$. In terms of $y$, what are the upper and lower bounds on the number of students we can assign to class C? 

**A:** <font color='blue'>Upper bound = $17y$; lower bound = $6y$.</font>

Below, you'll see a modified version of the <code>Assign</code> function. 

In [ ]:
# A modified FWS assignment model, which incorporates minimum class size and the option of not running class sections

# INPUTS:
# students: a list of students
# classes: a list of classes
# edges: a dictionary of edge costs
# minstudents: minimum number of students per class section
# csize: maximum number of students per class section
def modifiedAssign(students, classes, edges, minstudents, csize):
    STUDENT = students              # create student list
    CLASS = classes                 # create class list
    EDGES = list(edges.keys())      # create edge list    
    
    c = edges.copy()                # define c[i,j]
    
    # define model
    m = OR.Solver('assignFWS', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        # define x(i,j) here
        x[i,j] = m.IntVar(0, m.infinity(), ('(%d, %s)' % (i,j))) 
        
    y = {}
    for j in CLASS:
        # define y_j here
        y[j] = m.BoolVar('y_%s' % j) # A BoolVar or Boolean variable is similar to an integer variable,
                                     # except that it can only take on values in {0,1}, where 0 represents "false"
                                     # and 1 represents "true." We could have also used an IntVar ranging from 0 to 1.
               
    # define objective function here
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
   
    # add constraint to ensure each student is assigned exactly one class
    for k in students:
        m.Add(sum(x[i,j] for i,j in EDGES if i==k) == 1)
        
    # add constraint to ensure each class that runs satisfies minimum and maximum class size
    for k in classes:
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) <= csize*y[k])
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) >= minstudents*y[k])
    
    # solve
    status = m.Solve()

    if status == OR.Solver.INFEASIBLE:
        print('Infeasible')
        return

    print('All students matched.')
    
    classes_run = int(sum(y[j].solution_value() for j in CLASS))
    print(str(classes_run) + ' of ' + str(len(CLASS)) + ' classes run.')
    
    matched = {}
    for i,j in EDGES:
        if x[i,j].solution_value() == 1:
            if c[i,j] in matched:
                matched[c[i,j]] += 1
            else:
                matched.update({c[i,j] : 1})
    
    return matched       

Take a careful look at the updates made to the decision variables and constraints. Do they make sense based on what you answered in the previous series of questions? (If you're confused, ask a TA before moving on!)

**Q15:** In our previous <code>Assign</code> function, we added a constraint to ensure that each student (besides the dummy) was assigned at most one class. In our <code>modifiedAssign</code> function, we change this to strict equality: now each student is assigned exactly one class. Why? (Hint: think about how our new decision variables impact the cost of a solution)

**A:** <font color='blue'>If we did not update this constraint, the solver will opt to not assign any students by setting every $x[i,j]$ equal to zero (by way of setting every $y[k]$ equal to zero). That is, the cheapest solution (of cost 0) is to not run any classes!</font>

One advantage of our modified assignment function is that it's flexible in the case that a class must be canceled for some reason. For instance, if the Knight Institute wanted to cancel the least popular class (section 1), we'd just put the following line of code in with our constraints: 

<code>m.Add(y[1] == 0)</code>

Now, let's test out our new model! Run the cell below, which inputs the Knight Institute's constraints by setting the parameter 'minstudents' equal to 6 and 'csize' equal to 17. As a reminder, in our min-cost flow formulation (that is, before we included the ability to not run classes), this input returned "Infeasible."

In [ ]:
modified_sol = modifiedAssign(students, classes, edges, 6, 17)
print(modified_sol)

Woohoo! We found a feasible solution! 

Let's see how it compares to the original model, which had no minimum class size.

In [ ]:
from fws_lab_ext import Histo

print("Assign function solution: no minimum class size")
Histo(data_sol, 15)

for pref in range(1,6):
    print(pref, ":", round(100*data_sol[pref]/2285, 2), "%")

print("")
    
print("Modified Assign function solution: minimum class size 6")
Histo(modified_sol, 15)

for pref in range(1,6):
    print(pref, ":", round(100*modified_sol[pref]/2285, 2), "%")

**Q16:** Compare the objective values of these two solutions (you can use the cell below for computations if you'd like). What do you observe? Does this make sense? (Remember we set the cost of a student receiving their $k$th preference to be $k$; that is, a student receiving their top choice cost 1, second choice cost 2, and so forth. Ignore any contributions from dummy edges in the first solution.)

**A:** <font color='blue'>The first (less constrained) solution has objective value $(1179)(1) + (634)(2) + (301)(3) + (116)(4) + (55)(5) = 4089$, and the second (more constrained) solution has objective value $(1188)(1) + (616)(2) + (302)(3) + (126)(4) + (53)(5) = 4095$. This should make sense, as adding more constraints to the model (i.e., shrinking the feasible region) can only maintain or worsen the objective value.</font>

In [ ]:
### CELL FOR COMPUTATIONS ###


**Q17:** The Knight Institute is worried about money and so wants to know the least amount of classes they can offer while still having students get one of their top 5 preferences. To implement this, we can re-define the objective function in our model to the following:

<code>m.Minimize(sum(y[j] for j in CLASS))</code>

Doing this will give a feasible solution with an objective value of 135. We claim that this is optimal! Give an argument as to why they'll always need at least 135 sections. (Recall that there are 2285 students in total.)

**A:** <font color='blue'>The objective value essentially says that we can never run less than 135 class sections if we want a feasible solution. Suppose we fill each classroom to the max, that is, assign 17 students per class for as many classes as possible. We can fill $\frac{2285 - 2285\bmod17}{17} = 134$ classes this way, with a remaining $2285\bmod17 = 7$ students to put in the 135th class section.</font>

(The fact that we're able to find a feasible assignment with exactly 135 sections open may seem pretty magical. Disclaimer: this is indeed a bit magical, and may not always happen; we managed to get lucky with the data!)

**Q18:** Do you expect this solution to be better or worse (for students) as the min-cost solutions above? Explain your answer. 

**A:** <font color='blue'>We'd expect this solution to be worse for students, because it just tries to minimize the number of sections run while each student gets one of their listed preferences (it doesn't matter which). In contrast, the min-cost solutions incorporate the extra step of ranking of students' preferences once a feasible solution (where each student gets one of their listed preferences) is guaranteed.</font>

Run the cell below to get a visual of the "least-classes" solution!

In [ ]:
from fws_lab_ext import minimizeNumClassesAssign
least_classes_sol = minimizeNumClassesAssign(students, classes, edges, 6, 17)
Histo(least_classes_sol,15)

for pref in range(1,6):
    print(pref, ":", round(100*least_classes_sol[pref]/2285, 2), "%")

Clearly, there's a trade-off here: by running fewer classes, the university saves money, but doing so might leave some students dissatisfied with their FWS assignment, and vice-versa. This is a peek into the messy world of *multi-objective optimization*! 

In a sense, we want the "best of both worlds"&mdash;that is, we want to both minimize the number of sections run and minimize student dissatisfaction. We already know how to find each of these solutions individually, as well as what the solutions actually are: a section-optimal solution runs 135 sections, while a student-optimal solution has cost 4095. To find a good intersection between the two, one strategy is to "fix" one of these values (by way of constraints) and optimize the other as much as possible. (Simplex, anyone?)

**Q19:** For our problem, we have two strategies: either fix the number of sections and minimize cost, or fix the cost and minimize number of sections. Give a reason why we might choose the first strategy.

**A:** <font color='blue'>Trying to decrease the number of sections after fixing the cost (which, as we'll see, is just an arbitrary number) probably won't give us anything interesting, because decreasing the number of sections should just increase the cost.** On the other hand, there are likely several solutions (with varying costs) that run the same number of sections. 

** Sam made a good point that we could write an IP like "How few sections can I have while staying within 5% as good a solution?", which would motivate the second strategy.</font>

**Q20:** Write the constraint that our model must run exactly 135 classes, using the code from **Q17** as a guide. Your answer should look like this: m.Add(XXX)<br>(Don't worry, you don't need to paste your answer anywhere.)

**A:** <font color='blue'><code>m.Add(sum(y[j] for j in CLASS) == 135)</code></font>

In the cell below, we run the <code>modifiedAssign</code> function with the constraint you wrote above. Then we vary the number of classes that are run, minimizing cost (and thus student dissatisfaction) at each step. 

Look at the scatter plot. It compares the cost of these solutions with the cost of the student-optimal solution&mdash;the closer the cost, the better. Does this surprise you?

In [ ]:
from fws_lab_ext import modifiedAssignWithNumClasses

# Helper function: return the objective value of a solution dictionary of the form {edge cost : number of edges in solution}
def objValue(sol):
    keys = list(sol.keys())
    vals = list(sol.values())
    return sum(keys[i]*vals[i] for i in range(len(keys)))

# Find optimal solutions when exactly 135, 136, ..., 141 class sections are running
obj_values = {}
for i in range(135,142):
    print(str(i) + " classes offered:")    
    class_sol = modifiedAssignWithNumClasses(students, classes, edges, 6, 17, i)    
    if class_sol != None:
        obj_values[i] = objValue(class_sol)
        print("Objective value " + str(objValue(class_sol)))
    print("")
    
# Display scatter plot of solution values compared to optimal solution value
sectionsOffered = list(obj_values.keys())
pctOptimal = [100 * objValue(modified_sol) / x for x in list(obj_values.values())]
plt.scatter(sectionsOffered, pctOptimal)
plt.xlabel('Number of Sections Offered')
plt.ylabel('Optimality Metric (%)')
plt.show()

Take-homes from this section:
* By updating our integer program model, we accounted for "real-world" scenarios: both a minimum class size and the possibility of cancelling classes. (Adding the flexibility to cancel class sections happened to let us turn an infeasible problem into a feasible one!)
* With multiple feasible solutions, we may need to consider the trade-offs that come with prioritizing different objectives.

# Part 3: New Objectives
*Adapted from previous versions of the FWS lab.* 

As you have probably noticed, there is more than one feasible solution to the FWS assignment problem&mdash;that is, there exist multiple (potentially many) matchings of students to FWS sections that ensures every student gets one of their top 5 picks. Having many solutions is great, but in a real-life scenario, we eventually have to pick one!

Once we know we have feasible solutions, deciding which of the solutions is "the best" depends on what our goal is. For instance, our goal could be to find the solution that maximizes the number of students receiving their first choice, or to minimize the number of students receiving their fifth choice.

It turns out we can achieve some complex behavior in our solutions by simply making some clever adjustments to our original objective function.

**Q21:** In our original input for the FWS assignment problem, we set the unit cost of edges from the dummy node to be an absurdly high number (100,000). Why? 

**A:** <font color='blue'>In the FWS input, edge costs dictate how much you want the solver to select the corresponding edges. Since we are trying to minimize cost, an edge with a small cost has a higher likelihood of being in the solution while an edge with a large cost will potentially be avoided. (For instance, we want more first-choice than fifth-choice, so the cost of a first-choice edge is lower than the cost of a fifth-choice.) Applying this to the dummy, setting the edge cost to an enormous number like 100,000 essentially discourages the solver from ever choosing a dummy edge over a real student edge unless absolutely necessary to get a feasible solution. 

More technically, the cost of not assigning just *one* student to one of their top 5 choices (i.e., the cost of assigning a dummy student in place of a real student) is greater than the cost of assigning *all* 2285 students their fifth choice: $100,000 > 5(2285) = 11,425$. So we'd rather assign every student, if such an assignment exists, than fail to assign just one student! Using this strategy thus ensures we maximize the number of assigned students first before considering any notion of student preference.</font>

Let's use this notion of "weight adjustment" to see how it plays into the solutions we come up with. Run the cell below, which creates a function to facilitate incorporating varied edge costs into our modified integer program. 

In [ ]:
from fws_lab_ext import updated_edge_costs

def part3(prefcosts):
    # Get the updated edge : cost dictionary
    newedges = updated_edge_costs(edges,prefcosts)

    # Assign students based on these new edge costs
    part3_sol = modifiedAssign(students, classes, newedges, 6, 17)

    # Format results
    new_costs_to_prefs = dict(zip(list(prefcosts.values()),list(range(1,6))))

    prefs_to_nums = {}
    for key in part3_sol.keys():
        prefs_to_nums[new_costs_to_prefs[key]] = part3_sol[key]   
    
    # See histogram
    Histo(prefs_to_nums,15)
    
    for pref in range(1,6):
        if pref in prefs_to_nums.keys():
            print(pref, ":", round(100*prefs_to_nums[pref]/2285, 2), "%")
        else:
            print(pref, ":", "0 %")

**Q22:** If we decide to update the edge costs, what changes in our integer program? 

**A:** <font color='blue'>The cost matrix $c[i,j]$ will be updated with the new unit costs.</font>

Going back to our example, let's find a solution that maximizes the number of students receiving their first choice. To do this, we need to somehow "entice" the solver into picking edges corresponding to students' first preference. This is just the opposite of what we did with the dummy node! 

In the cell below, we set the cost of edges between a student and their first-choice class to be an arbitrarily large *negative* number (-10,000). It's worth noting that edge costs can be positive or negative here, since we're not so much concerned about the actual cost of our solution as we are about the relationship between the costs of different types of edges.

In [ ]:
# Define new edge costs here
prefcosts = {1:-10000,
             2:2,
             3:3,
             4:4,
             5:5}

# Run the model with the new edge costs
part3(prefcosts)

**Q23:** Just looking at the histogram, how does this solution compare to the original solution we found? (How does the number of first choices compare? Fifth choices?)

**A:** <font color='blue'>This solution does better in terms of the number of first choices received (more than original), but worse in the number of fifth choices received (more than original).</font>

Next, let's see a solution where the number of students receiving their fifth choice is minimized. 

In the cell below, modify the cost of edges from a student to their fifth-choice class to achieve this objective, then run it and see what you get! 

In [ ]:
# Define new edge costs here
prefcosts = {1:1,
             2:2,
             3:3,
             4:4,
             5:5   ### CHANGE THIS VALUE ###
            }

# Run the model with the new edge costs
part3(prefcosts)

**Q24:** Compare these two solutions (maximizing first choice versus minimizing fifth choice). If you had to present one of these solutions to the FWS assignment committee, which would you present? Give a reason for your choice.

**A:** <font color='blue'>Answers may vary. One justification for the first solution is that nearly half the students receive their top choice. One justification for the second solution is that it has a lesser (meaning better) mean preference received, and no student receives their last choice (out of their top 5 choices, of course!).</font>

What if we wanted to combine these approaches? For example, we could first maximize the number of students receiving their first preference, then maximize the number of students receiving their second preference, and so on. Or, we could first minimize the number of students receiving their fifth choice, then minimize the number of students receiving their fourth choice, etc. In both cases, there exists a notion of "ranking" or "prioritizing" different objectives.

How can we encode a ranking of importance among our different objectives? We'll use weights of different orders of magnitude.

For example, suppose we have two objective functions, $f_{1}$ and $f_{2}$. We'd like to first minimize $f_{1}$, and then minimize $f_{2}$. If we were to write out an expression such as $min\:f_{1} + f_{2}$, then decreasing either $f_{1}$ or $f_{2}$ by some amount $\delta$ would have the same effect; in essence, $f_{1}$ and $f_{2}$ have "equal priority."

However, if instead we write $min\:10f_{1} + f_{2}$, then decreasing $f_{1}$ by $\delta$ has 10 times the effect on lowering the cost of the solution as decreasing $f_{2}$ by the same amount&mdash;so minimizing $f_{1}$ has "greater priority" than minimizing $f_{2}$. 

We can apply similar reasoning to our problem; in fact, we already have! Take another look at our objective function:

$min\:\sum_{(i,j) \in A} c(i,j)x(i,j) = min\:\sum_{k=1}^{5} ($edge cost of preference $k)($number of students assigned preference $k)$<br>$= min\:c_1 f_{1} + c_2 f_{2} + c_3 f_{3} + c_4 f_{4} + c_5 f_{5}$

Our costs $c_k$ for each edge type $k$, enumerated in <code>prefcosts</code>, specify the "weights" (or "priorities") of each function $f_k$ in our overall objective function. For instance, specifying costs $c_k = k, k \in \{1,..,5\}$, prioritizes minimizing $f_{5}$ before minimizing $f_{1}$. 

In [ ]:
## Begin Sam example

**Example**

Suppose we have 9 students to assign to FWS sections, and each student lists 4 preferences. We want to minimize the number of students receiving their fourth choice, then third choice, and so on. 

Let's set <code>prefcosts</code> so that the cost of each preference is a multiple of 10:

<code>prefcosts = {1:1, 2:10, 3:100, 4:1000}</code>

We might wonder if the IP solver could still assign someone their fourth choice in order to reduce the cost: for instance, maybe this allows us to give a lot more students their first choice. But this is impossible!

Why? Consider the cost of an assignment where all 9 students are assigned to their first or second or third choice, that is, no students are assigned their fourth choice. This will cost at most $9(100) = 900 < 1000$ (the cost of assigning just one student their fourth choice). Thus the IP solver will never opt to assign someone their fourth choice unless this is the only way to achieve a feasible solution&mdash;otherwise, it's just too expensive. 

Using the same reasoning, you can justify that the IP will then minimize the number of third-choice assignments (and so on).

More generally, if we have two objectives (e.g., number of third-choice and number of fourth-choice) that can take on values in the range $\{0,...,9\}$, then multiplying the weight of one objective by 10 would make it completely dominant over the other.

Thus, by cleverly choosing our edge costs, we can make it so that our higher-priority objectives 'dominate' lower-priority objectives. This is known as a *lexicographic ordering*.

**Q:** Would this approach (using multiples of 10) still work if we had 10 students to assign?

**A:** <font color=blue>Yes. Consider the minimum cost of a solution in which at least one student is assigned their fourth choice: $9(1) + 1000 = 1009$. Now compare this to the maximum cost of a solution in which no student is assigned their fourth choice: $10(100) = 1000 < 1009$. So the solver will still never choose assigning a student their fourth choice if it can be avoided. </font> 

**Q:** Would this approach (using multiples of 10) still work if we had 11 students to assign?

**A:** <font color=blue>No. For example, assigning 10 students their first choice and 1 student their fourth choice costs 1010, and assigning 1 student their second choice and 10 students their third choice would also cost 1010. So the solver might choose to assign a fourth choice when it could've avoided it.</font>

**Q:** Now imagine we have $n$ students to assign. By what factor should the objective weights differ to ensure we get the ordering we want?

**A:** <font color=blue>By a factor of $n$.</font>

In [ ]:
## End Sam example

Let's utilize this knowledge to implement the second approach outlined above.

Run the following cell, which uses weights of different orders of magnitude to effectively "rank" our objectives in order of importance: first (most importantly), we minimize the number of fifth preferences assigned, second (most importantly), we minimize the number of fourth preferences assigned, and so forth until our least important objective, minimizing the number of first preferences.

In [ ]:
# Define new edge costs here
prefcosts = {1:1,
             2:10,
             3:100,
             4:1000,
             5:10000}

# Run the model with the new edge costs
part3(prefcosts)

As we've seen, there are a plethora of solutions to the FWS assignment problem! Choosing just one depends on how you define the "best" solution. 

The Knight Institute decided on the following criteria in determining the optimal solution:
* First, the number of fifth, and then fourth, preferences should be minimized.
* After that, the number of first, second, and then third preferences should be maximized.

Below, try to implement these specifications, using weights with different orders of magnitude for each successive objective. (Hint: minimizing a negative expression is the same as maximizing that expression!)

In [ ]:
# TODO: Fill in the missing edge costs to achieve the desired objectives
# prefcosts = {1:XXX, 2:XXX, 3:-1, 4:XXX, 5:10000}

### BEGIN SOLUTION
prefcosts = {1:-100, 2:-10, 3:-1, 4:1000, 5:10000}
### END SOLUTION

# Run the model with the new edge costs
part3(prefcosts)

**Q25:** Compare this solution to the previous solutions. Do you think the Knight Institute made the right decision? How would you have done it differently?

**A:** <font color='blue'>Answers may vary.</font>

### Supplemental Exercise

In each of the previous models, we set the weights/unit costs of different types of edges at different orders of magnitude to essentially order our objectives. However, an order of magnitude seems pretty arbitrary. Can we be more precise in how we set our edge weights in order to prove the model is giving us the solution we want?

Let's consider a simple example. Say that we have 50 students that must be assigned. Each student provides two preferences, so in a solution they are either (a) matched with their first preference, (b) matched with their second preference, or (c) left unmatched. We want to prove that our model will minimize the number of unmatched students. 

Suppose that matching a student with their first preference incurs a (nonnegative) cost $w_a$, matching a student with their second preference costs $w_b$, and not matching a student costs $w_c$. We want to minimize unmatched, then minimize second preference. It makes intuitive sense that we must order our costs so that $0 \leq w_a < w_b < w_c$.

First, let's try using our order-of-magnitude strategy from before. We set $w_a = 1$, $w_b = 10$, and $w_c = 100$. 

Suppose there are two feasible solutions: Solution 1 assigns 5 students their first choice and 45 their second choice, while Solution 2 assigns 46 students their first choice and leaves 4 students unmatched.

**Q26:** Does our order-of-magnitude strategy work here? Explain.

**A:** <font color='blue'>We want the solution with less unmatched students (Solution 1), but with our current edge weights, the solver will pick Solution 2, which has cost 400 (compared to 455 for Solution 1). So our strategy doesn't work in this case.</font>

Let's see if we can set our edge costs a little better.

**Q27:** In terms of $w_a$, $w_b$, and $w_c$, give an upper bound on the cost of a solution that leaves no students unmatched.

**A:** <font color='blue'>If every student is matched, the most costly scenario is that all 50 students are assigned their second choice, since $w_a < w_b$. The cost of the solution is then $50w_b$.</font>

**Q28:** In terms of $w_c$ alone, give a lower bound on the cost of a solution that does not match every student. (Hint: make $w_a$ as small as possible)

**A:** <font color='blue'>Since $w_a < w_b < w_c$, the cheapest solution that does not match all 50 students is the one that matches 49 students with their first choice and leaves 1 student unmatched. This has cost $49w_a + w_c$. However, an even better lower bound assumes $w_a$ is minimized, i.e., $w_a = 0$. Then the cost is simply $w_c$.</font>

**Q29:** Using your answers to the previous two questions, write an inequality that ensures our solution minimizes the number of unmatched students.

**A:** <font color='blue'>We want to set our weights such that the worst solution that matches all students is still better than a solution that leaves a student unmatched. So we simply combine our expressions from above: 

$50w_b < w_c \iff \frac{w_c}{w_b} > 50$
</font>

**Q30:** Now suppose there are $n$ students. By what factor should the edge weights $w_b$ and $w_c$ differ to guarantee the number of unmatched students is minimized? 

**A:** <font color='blue'>By a factor of $n$; more specifically, $w_c > n \cdot w_b$.</font>

To summarize, we can set our edge weights in such a way so that the cost of assigning all of our $n$ students to a class section is less than the cost of leaving just *one* student unmatched! This guarantees that whatever solution the solver returns is minimizing the number of students unmatched before considering anything else.

More generally, we can ensure our model focuses on our highest-priority objective first by setting our edge weights so that any solution that does better in terms of this objective is valued more than a solution that does worse in terms of this objective, regardless of how the solutions compare for lower-priority objectives. 

In multi-objective optimization, this strategy of ranking objectives (and objective functions) is referred to as the *lexicographic method*. We've simulated this by assigning different weights to the various smaller objective functions (minimize unmatched, minimize second-choice, minimize first-choice) and summing them up into a formulation with a single objective function, which is known as the *weighted-sum method*. 

**Q31:** Suppose we know there exist solutions where every student can be matched. By similar reasoning to that described above, propose a relationship between $w_a$ and $w_b$ that ensures the number of students receiving their second choice is minimized. 

**A:** <font color='blue'>$n \cdot w_a < w_b \iff \frac{w_b}{w_a} > n$</font>

Let's apply what we've learned to the actual FWS input. Suppose we want to first minimize the number of students receiving their fifth preference, then minimize the number of students receiving their fourth preference, and so on. Fill in the missing edge costs (aka objective function weights!) below that will ensure we meet our goal.

In [ ]:
n = len(students)

# TODO: Fill in the missing edge costs to achieve the desired objectives.
# Hint: For exponentiation, we use the ** operator. For example, to calculate 2^10, we'd type 2**10.

# prefcosts = {1:1, 2:XXX, 3:XXX, 4:n**3, 5:XXX}

### BEGIN SOLUTION
prefcosts = {1:1, 2:n, 3:n**2, 4:n**3, 5:n**4}
### END SOLUTION

In [ ]:
# Optional: run the model with the new edge costs as an initial sanity check
part3(prefcosts)

**Q32:** An 1101 student sees the $n^4$ term in the edge costs above and is worried the computer they're using won't be able to handle such a big number. They decide to divide all the edge costs by $n^2$, so that the largest edge cost is now only $n^2$. Will this give the same solution? Explain.

**A:** <font color='blue'>Yes! Scaling the edge costs by any scaling factor $k > 0$ preserves the relationship between them (that consecutive edge types' costs differ by a factor of $n$).</font>

**Q33:** Another 1101 student sets <code>prefcosts</code> to the following:

<code>prefcosts = {1:0, 2:1, 3:n, 4:n\*\*2, 5:n\*\*3}</code>

Will this give the solution we want? Explain.

**A:** <font color='blue'>Yes! Comparing preferences 2-3, 3-4, and 4-5, the factor of $n$ separating the respective edge costs creates the "ranking" we want. For preferences 1-2, setting the cost of first-choice edges to 0 makes it "infinitely" more expensive to assign a student their second choice than to assign all $n$ students their first choice, since $\frac{w_2}{w_1} = \lim_{k \to 0}\frac{1}{k} = \infty$. Since $\infty > n$, this is fine.</font>

# Part 4: A Note on Integrality

The integer program we've developed so far has integer decision variables $x(i,j) \in \{0, 1, ...\}$, corresponding to the amount of flow to send on an arc $(i,j)$ from a student node $i$ to a class node $j$. As an improvement, we added new binary decision variables $y(j) \in \{0, 1\}$ that encode whether class $j$ runs or not. 

What happens if we remove ("relax") our integrality constraints on $x(i,j)$? (This is known as an *LP relaxation* of the integer program.) To find out, we'll use the following function, which is pretty much the same as the <code>modifiedAssign</code> function we've been using, except we can specify whether our $x(i,j)$ are restricted to only taking on integer values.

In [ ]:
# Same as function modifiedAssign, but with optional additional parameter:
# integer_only: Boolean value (True/False), whether or not decision variables x[i,j] are constrained to integers (default: True)
def integralityAssign(students, classes, edges, minstudents, csize, integer_only = True):
    STUDENT = students              # create student list
    CLASS = classes                 # create class list
    EDGES = list(edges.keys())      # create edge list    
    
    c = edges.copy()                # define c[i,j]
    
    # define model
    m = OR.Solver('assignFWS', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        # define x(i,j) here
        if integer_only:
            x[i,j] = m.IntVar(0, m.infinity(), ('(%d, %s)' % (i,j)))
        else:
            x[i,j] = m.NumVar(0, m.infinity(), ('(%d, %s)' % (i,j)))
  
    print('Decision variables constrained to integer values.' if integer_only else 'Decision variables unconstrained.')
    
    y = {}
    for j in CLASS:
        # define y_j here
        y[j] = m.BoolVar('y_%s' % j) # A BoolVar or Boolean variable is similar to an integer variable,
                                     # except that it can only take on values in {0,1}, where 0 represents "false"
                                     # and 1 represents "true." We could have also used an IntVar ranging from 0 to 1.
               
    # define objective function here
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
       
    # add constraint to ensure each student is assigned exactly one class
    for k in students:
        m.Add(sum(x[i,j] for i,j in EDGES if i==k) == 1)
        
    # add constraint to ensure each class that runs satisfies minimum and maximum class size
    for k in classes:
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) <= csize*y[k])
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) >= minstudents*y[k])
    
    ### ANY OTHER CONSTRAINTS HERE
    #m.Add(sum(x[i,j] for i,j in EDGES if c[i,j] == 1000) <= 102)
    ###
    
    # solve
    status = m.Solve()
    
    if status == OR.Solver.INFEASIBLE:
        print('Infeasible')
        return
     
    print('All students matched.')
    
    # test for non-integer values
    if not integer_only:
        nonint = False
        for i,j in EDGES:
            sol = round(x[i,j].solution_value(),2)
            if sol > 0.0 and sol < 1.0:
                # print(i,j,sol)
                nonint = True
        print('Non-integer values in solution.' if nonint else 'Only integer values in solution.')
        print('') 
    
    matched = {}
    for i,j in EDGES:
        sol = round(x[i,j].solution_value(),2)
        if sol > 0.0:
            if c[i,j] in matched:
                matched[c[i,j]] += sol
            else:
                matched.update({c[i,j] : sol})
    
    return matched

Additionally, let's add the ability to modify edge costs and print the results using the function below:

In [ ]:
def part4(prefcosts, integeronly = True):
    
    # Get the updated edge : cost dictionary
    newedges = updated_edge_costs(edges,prefcosts)

    # Assign students based on these new edge costs
    test_sol = integralityAssign(students, classes, newedges, 6, 17, integer_only=integeronly)

    if test_sol == None: # infeasible
        return
    
    # Format results
    new_costs_to_prefs = dict(zip(list(prefcosts.values()),list(range(1,6))))

    prefs_to_nums = {}
    for key in test_sol.keys():
        prefs_to_nums[new_costs_to_prefs[key]] = test_sol[key]   
    
    # See histogram
    Histo(prefs_to_nums,15)
    
    for pref in range(1,6):
        if pref in prefs_to_nums.keys():
            print(pref, ":", round(100*prefs_to_nums[pref]/2285, 2), "%")
        else:
            print(pref, ":", "0 %")
    print('')

The Knight Institute saw our graphs from Part 3 and decided they still liked their approach the best: minimizing the number of students receiving their fifth choice, then minimizing fourth choice, then maximizing first, second, and then third choice. As a refresher, run the cell below to see what that solution looks like. (By setting the parameter 'integeronly' to <code>True</code>, we tell the model to create the decision variables as integer variables.)

In [ ]:
# Define new edge costs here
prefcosts = {1:-100,
             2:-10,
             3:-1,
             4:1000,
             5:10000}

# Run the model with the new edge costs
integer_only_sol = part4(prefcosts, integeronly = True)

If all goes well, you should get the same solution you got in Part 3. 

Now, let's see if anything interesting happens when our $x(i,j)$ are no longer confined to the happy world of integers! We do this by setting our parameter 'integeronly' to <code>False</code>.

In [ ]:
not_only_integer_sol = part4(prefcosts, integeronly = False)

Hmm, doesn't seem like much changed. Maybe we can just use our LP-relaxation from now on, since it's more efficient and cheaper to use an LP solver than an IP solver.

The Knight Institute sees our solution and wants to know if we can drop the number of students with their fourth choice by one, from 103 to 102, so that the total percentage of students with their top three choices is above 96.5% (an A+!) 

To see if this is possible, **uncomment the following constraint in the** <code>integralityAssign</code> **function**, which caps the number of students receiving their fourth choice at 102:

<code>m.Add(sum(x[i,j] for i,j in EDGES if c[i,j] == 1000) <= 102)</code>
    
Make sure you **run the cell containing the** <code>integralityAssign</code> **function** afterwards!

Once you've done so, run the cell below to see if we can do as the Knight Institute says.

In [ ]:
not_only_integer_sol_add_constraint = part4(prefcosts, integeronly = False)

Uh-oh...we broke integrality! It doesn't make much sense to be assigning half a student their fifth-choice FWS preference.

**Q34:** The Knight Institute isn't too keen on splitting students in half. They argue that there must be some way to assign whole (integer) students where (a) no student gets their fifth-choice preference, (b) 102 students (or less) get their fourth-choice preference, and (c) the rest get one of their top three preferences. Explain why this is impossible.

**A:** <font color='blue'>To satisfy this, we need an integer solution that is feasible (i.e., matches all students), has 102 (or less) fourth-choice, and has 0 fifth-choice. Such a solution would have to be better than the one we just found, which has 0.5 fifth-choice. But we know that we cannot find an integer solution better than this LP-relaxation solution, since the IP-feasible region is contained within the LP-feasible region (and thus the best solution for the IP can be no better than the best solution for the corresponding LP-relaxation).</font>

You may be wondering why adding this constraint in particular broke integrality. As it turns out, we're lucky that none of the previous constraints we added broke it first! 

The moral of the story is that we can't rely on an LP relaxation if we're looking for integral solutions. Instead, we need to use an IP formulation and an IP solver. 